In [1]:
#import modules and read file into dataframe
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from openpyxl.workbook import Workbook

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

pd.options.display.max_columns=999
pd.options.display.max_rows=3000

data = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)#import modules and read file into dataframe
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
from openpyxl.workbook import Workbook

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

pd.options.display.max_columns=999
pd.options.display.max_rows=3000

data = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

In [3]:
df = data

In [4]:
#assigns 'Call Detail' table to dataframe
call_data = df['Call Details']
call_data.tail()#assigns 'Call Detail' table to dataframe
call_data = df['Call Details']
call_data.tail()

,month,Application_Id,Login_ID,Right_Party_Contact,Promise_to_pay,total_contacts
79609,5,MX-999897229,AGNT19,0,0,2
79610,5,MX-999911711,AGNT6,0,0,1
79611,5,MX-999978990,AGNT43,2,0,2
79612,5,MX-999980626,AGNT21,1,1,1
79613,5,MX-999985403,AGNT19,5,1,6


In [5]:
#groups call data by id and then aggregates
month_table = call_data.groupby(['Application_Id']).aggregate({'month':'nunique', 'Right_Party_Contact':'sum', 'Promise_to_pay':'sum', 'total_contacts':'sum'})

month_table


,month,Right_Party_Contact,Promise_to_pay,total_contacts
Application_Id,,,,
MX-100040250,1,5,2,21
MX-10005082,1,0,0,2
MX-100062359,1,1,1,1
MX-100095892,1,1,0,2
MX-100116696,3,5,2,10
...,...,...,...,...
MX-999916801,1,5,0,9
MX-999941023,1,5,1,155
MX-999978990,1,2,0,2


In [7]:
#calculates Right Party Contacts (RPC) rate and Percentage of outbound calls resulting in Promise to Pay (PTP)
# month_table['avg_intensity'] = round(month_table['total_contacts'] / month_table['month'], 3)

month_table['RPC_rate'] = round(month_table['Right_Party_Contact'] / month_table['total_contacts'], 3)

month_table['PTP_rate'] = round(month_table['Promise_to_pay'] / month_table['total_contacts'], 3)

month_table.rename(columns={'month':'month_count'}, inplace=True)

month_table

,month_count,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate
Application_Id,,,,,,
MX-100040250,1,5,2,21,0.238,0.095
MX-10005082,1,0,0,2,0.000,0.000
MX-100062359,1,1,1,1,1.000,1.000
MX-100095892,1,1,0,2,0.500,0.000
MX-100116696,3,5,2,10,0.500,0.200
...,...,...,...,...,...,...
MX-999916801,1,5,0,9,0.556,0.000
MX-999941023,1,5,1,155,0.032,0.006
MX-999978990,1,2,0,2,1.000,0.000


In [9]:
len(month_table)

38051

In [10]:
df.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

In [11]:
data = pd.merge(df['Loan_ID mapping'], month_table, how='outer', left_on='Application_id', right_on='Application_Id')

In [16]:
valid = len(month_table)
missing = len(data[data['total_contacts'].isnull()])

print((valid+missing)==len(data))

True


In [13]:
len(data)

40339

In [14]:
data[data['total_contacts'].isnull()]

,Loanid,Application_id,month_count,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate
6,A589318509,MX-100124165,NaN,NaN,NaN,NaN,NaN,NaN
18,A518359587,MX-100429071,NaN,NaN,NaN,NaN,NaN,NaN
24,A123751865,MX-100513872,NaN,NaN,NaN,NaN,NaN,NaN
36,A375423985,MX-100650190,NaN,NaN,NaN,NaN,NaN,NaN
43,A622387745,MX-100781834,NaN,NaN,NaN,NaN,NaN,NaN
61,A354198119,MX-101088368,NaN,NaN,NaN,NaN,NaN,NaN
74,A18116222,MX-101299701,NaN,NaN,NaN,NaN,NaN,NaN
85,A381592980,MX-101481600,NaN,NaN,NaN,NaN,NaN,NaN
126,A327294090,MX-102087769,NaN,NaN,NaN,NaN,NaN,NaN
134,A367528674,MX-102254007,NaN,NaN,NaN,NaN,NaN,NaN
